### Import

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
import torchvision
from torchmetrics.functional import precision_recall, accuracy, f1_score
from torch.utils.tensorboard import SummaryWriter
import cv2

### Data Info

In [2]:
# Learning data directory
DATASET_NAME = 'dataset20220823'
DATA_ROOT = '/export/hashimoto/DATA/' + DATASET_NAME + '/'
JSON_FILE = DATA_ROOT + 'dataset.json'
IMAGE_DIR = DATA_ROOT + 'img/'
SLICE_NUM = 3
IMAGE_SIZE = 128

### Params

In [3]:
VALID_SIZE = 40/176
GAUSSIAN_SIGMA = 5

WINDOW_SIZE = 15
WINDOW_STRIDE = 3
WINDOW_SIGMA = 10

VMIN = 240
VMAX = 255

PROJECT = 'A6c'
LOAD_DIR = '/export/hashimoto/Results/' + PROJECT + '/'
LOAD_FILE = '_model_best.pth'
SAVE_DIR = '/export/hashimoto/ProbabilityMap/' + PROJECT + '/'

# Select using GPU
device = torch.device('cuda:0')

### Directry and Log files

In [4]:
# Project name (using for save and log directory name)
FOLDER = 'A6c'
SAVE_DIR = '/export/hashimoto/ProbabilityMap/' + FOLDER + '/'
os.makedirs(SAVE_DIR, exist_ok = True)

### Data List

In [5]:
with open(JSON_FILE, 'r') as f:
    jsondata = json.load(f)

files, labels = [], []
for key in jsondata.keys():
    info = key
    t2, adc, t1d = jsondata[key]['T2'], jsondata[key]['ADC'], jsondata[key]['T1D']
    
    files.append({'info':info, 't2':t2, 'adc':adc, 't1d':t1d})
    labels.append(jsondata[key]['ROI'])

# splid data (mtabun mazatte simau)
valid_files, valid_labels = files[:int(VALID_SIZE*len(files))], labels[:int(VALID_SIZE*len(files))]
train_files, train_labels = files[int(VALID_SIZE*len(files)):], labels[int(VALID_SIZE*len(files)):]
del files, labels

# print data list
print('valid data:{} (last element:{})'.format(len(valid_files), valid_files[-1]['info']))

print('train data:{} (first element:{})'.format(len(train_files), train_files[0]['info']))

valid data:1600 (last element:49_1_9)
train data:5440 (first element:50_1_1)


### Dataset

In [6]:
# int:0-255 [Numpy] => float:0.0-1.0 [Tensor]
class PixelToTensor(object):
    def __init__(self):
        pass
    def __call__(self, x):
        data = x / 255.0
        return torch.tensor(data, dtype = torch.float)
    
# fullImages (int:0-255 [Numpy]), windowCenter(int [array]) => maskImages (float:0.0-1.0 [Tensor])
class PositionToCropImages(object):
    def __init__(self):
        pass
    def __call__(self, fullImages, center):
        # 0 padding
        frame = torch.zeros(len(fullImages), len(fullImages[0]), IMAGE_SIZE+WINDOW_SIZE, IMAGE_SIZE+WINDOW_SIZE)
        frame[:,:,int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE,int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE] = fullImages
        # offset
        offsetCenter = center + int(WINDOW_SIZE/2)
    
        # crop range
        rangex = np.array([offsetCenter[0]-int(WINDOW_SIZE/2), offsetCenter[0]+int(WINDOW_SIZE/2)+1]).astype(int)
        rangey = np.array([offsetCenter[1]-int(WINDOW_SIZE/2), offsetCenter[1]+int(WINDOW_SIZE/2)+1]).astype(int)

        
        cropImages = frame[:, :, rangex[0]:rangex[1], rangey[0]:rangey[1]]

        return cropImages

class Dataset_CropImage(torch.utils.data.Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels
        self.pixelToTensor = PixelToTensor()

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        # info
        info = self.files[idx]['info']

        # exception
        if len(self.files[idx]['t1d']) != 20:
            print('info:' + info)
            print('t1dlen:' + str(len(self.files[idx]['t1d'])))

        # fullImages[volume][slice][x][y] voluem0:T2WI, volume1:ADC Map, volume2:T1WI, volume3:CE-T1WI
        fullImages = np.zeros((22, SLICE_NUM, IMAGE_SIZE, IMAGE_SIZE))
        for i in range(SLICE_NUM):
            fullImages[0][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['t2'][i]))
            fullImages[1][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['adc'][i])) 
            for j in range(20):     
                fullImages[2+j][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['t1d'][j][i]))
        fullImages = self.pixelToTensor(fullImages)

        output_size = int((IMAGE_SIZE + 2 * int(WINDOW_SIZE/2) - (WINDOW_SIZE-1) - 1) / WINDOW_STRIDE + 1)
        cropImages = torch.zeros(output_size, output_size, len(fullImages), len(fullImages[0]), WINDOW_SIZE, WINDOW_SIZE)
        
        # zero padding
        frame = torch.zeros(len(fullImages), len(fullImages[0]), IMAGE_SIZE+WINDOW_SIZE-1, IMAGE_SIZE+WINDOW_SIZE-1)
        frame[:,:,int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE,int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE] = fullImages

        for i in range(output_size):
            for j in range(output_size):
                cropImages[i,j] = frame[:,:,i*WINDOW_STRIDE:i*WINDOW_STRIDE+WINDOW_SIZE, j*WINDOW_STRIDE:j*WINDOW_STRIDE+WINDOW_SIZE]
        
        return info, cropImages

class Dataset_FullImage(torch.utils.data.Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels
        self.pixelToTensor = PixelToTensor()
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        # info
        info = self.files[idx]['info']
        
        # exception
        if len(self.files[idx]['t1d']) != 20:
            print('info:' + info)
            print('t1dlen:' + str(len(self.files[idx]['t1d'])))
        
        # fullImages[volume][slice][x][y] voluem0:T2WI, volume1:ADC Map, volume2:T1WI, volume3:CE-T1WI
        fullImages = np.zeros((22, SLICE_NUM, IMAGE_SIZE, IMAGE_SIZE))
        for i in range(SLICE_NUM):
            fullImages[0][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['t2'][i]))
            fullImages[1][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['adc'][i])) 
            for j in range(20):     
                fullImages[2+j][i] = np.array(Image.open(IMAGE_DIR + self.files[idx]['t1d'][j][i]))
        fullImages = self.pixelToTensor(fullImages)

        # convert MASK_IMAGE to ROI position(index)
        maskImage = np.array(Image.open(IMAGE_DIR + self.labels[idx]))
        maskImage = maskImage[:,:,1]
        roiCenter = np.array([0,0], int)
        for i in range(len(maskImage)):
            for j in range(len(maskImage[0])):
                if maskImage[roiCenter[0],roiCenter[1]] < maskImage[i,j]:
                    roiCenter = np.array([i, j], int)

        # generate gaussian roi map
        gaussianRoi = np.zeros((IMAGE_SIZE, IMAGE_SIZE))
        for i in range(IMAGE_SIZE):
            for j in range(IMAGE_SIZE):
                gaussianRoi[i, j] =  np.exp( -((roiCenter[0]-i)**2 + (roiCenter[1]-j)**2) / (2*((GAUSSIAN_SIGMA+1)**2)) )
        gaussianRoi = self.pixelToTensor(gaussianRoi*255.0)  
        
        return info, fullImages, gaussianRoi

### Network

In [7]:
class InputEncoder(torch.nn.Module):
    def __init__(self):
        super(InputEncoder, self).__init__()
        self.input = 0
        self.layer1 = 0
        self.output = 0
        
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU())
        
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU())
        
    def forward(self, input):
        self.input = input
        self.layer1 = self.conv1(self.input)
        self.output = self.conv2(self.layer1)
        return self.output 

class DynamicEncoder(torch.nn.Module):
    def __init__(self):
        super(DynamicEncoder, self).__init__()
        self.input = self.layer1 = self.output = 0
        
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv3d(in_channels = 1, out_channels = 32, kernel_size = (7,3,3), padding = (3,1,1), stride = (3,1,1)),
            torch.nn.BatchNorm3d(32), torch.nn.ReLU())
        
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv3d(in_channels = 32, out_channels = 32, kernel_size = (7,3,3), padding = (0,1,1), stride = 1),
            torch.nn.BatchNorm3d(32), torch.nn.ReLU())
    
    def forward(self, input):
        self.input = input
        self.layer1 = self.conv1(input)
        self.output = self.conv2(self.layer1)
        self.output = torch.squeeze(self.output)
        return self.output    

class GlobalEncoder(torch.nn.Module):
    def __init__(self):
        super(GlobalEncoder, self).__init__()
        self.input = 0
        # Encoder layer
        self.layer1 = self.layer2 = self.layer3 = 0
        # Decoder layer
        self.layer4 = self.layer5 = self.layer6 = 0
        self.positionToCropImages = PositionToCropImages()
        # output
        self.cropFeature = self.gaussian = 0
        
        self.step1 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2), torch.nn.Dropout(),
            torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(256), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(256), torch.nn.ReLU())

        self.step2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2), torch.nn.Dropout(),
            torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU())
        
        self.step3 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2), torch.nn.Dropout(),
            torch.nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(1024), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 1024, out_channels =1024, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(1024), torch.nn.ReLU())
        
        self.deconv1 = torch.nn.ConvTranspose2d(in_channels = 1024, out_channels = 512, kernel_size = 2, stride = 2)
        
        self.step4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 1024, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU(), torch.nn.Dropout())
        
        self.deconv2 = torch.nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 2, stride = 2)
        
        self.step5 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 512, out_channels = 256, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(256), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(256), torch.nn.ReLU(), torch.nn.Dropout())
        
        self.deconv3 = torch.nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 2, stride = 2)
        
        self.step6 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels =128+128, out_channels = 128, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(128), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(128), torch.nn.ReLU(), torch.nn.Dropout())
        
        self.step7 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 128, out_channels = 2, kernel_size = 1),
            torch.nn.ReLU(),
            torch.nn.Softmax(dim = 1))  
        
    def forward(self, fullImages, cropCenter):
        self.input = fullImages
        # Encode
        self.layer1 = self.step1(self.input)
        self.layer2 = self.step2(self.layer1)
        self.layer3 = self.step3(self.layer2)
        
        # Decode
        self.layer4 = self.step4(torch.cat([self.deconv1(self.layer3), self.layer2], dim = 1))
        self.layer5 = self.step5(torch.cat([self.deconv2(self.layer4), self.layer1], dim = 1))
        self.layer6 = self.step6(torch.cat([self.deconv3(self.layer5), self.input], dim = 1))

        # output
        self.cropFeature = torch.zeros(len(self.layer6), len(self.layer6[0]), WINDOW_SIZE, WINDOW_SIZE)
        for i in range(len(self.layer6)):
            self.cropFeature[i] = self.positionToCropImages(self.layer6[i].unsqueeze(dim=0), cropCenter[i])
        self.gaussian = self.step7(self.layer6)
        
        return self.cropFeature, self.gaussian[:,0]
        
class LocalEncoder(torch.nn.Module):
    def __init__(self):
        super(LocalEncoder, self).__init__()
        self.input = 0
        self.layer1 = self.layer2 = self.layer3 = self.layer4 = 0
        self.output = 0

        self.step1 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1), torch.nn.Dropout(),
            torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(512), torch.nn.ReLU())
        
        self.step2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2), torch.nn.Dropout(),
            torch.nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(1024), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels = 1024, out_channels = 256, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(256), torch.nn.ReLU())
        
        self.step3 = torch.nn.Sequential(
            torch.nn.Flatten(start_dim=1),
            torch.nn.Linear(256*4*4, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU())

        self.step4 = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU())

        self.step5 = torch.nn.Sequential(
            torch.nn.Linear(128, 2),
            torch.nn.Sigmoid(),
            torch.nn.Softmax(dim=1))
        
    def forward(self, cropImages): 
        self.input = cropImages
        self.layer1 = self.step1(self.input)
        self.layer2 = self.step2(self.layer1)
        self.layer3 = self.step3(self.layer2)
        self.layer4 = self.step4(self.layer3)
        self.output = self.step5(self.layer4)

        return self.output[:,0]
        
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.input = 0
        self.globalInputFeature = self.localInputFeature = 0
        self.globalDynamicFeature = self.localDynamicFeature = 0
        self.cropFeature = 0
        # output
        self.gaussian = self.output = 0
        
        # input encoder
        self.globalInputEncoder = [InputEncoder().to(device) for i in range(3)]
        self.localInputEncoder = [InputEncoder().to(device) for i in range(3)]
        self.globalDynamicEncoder = DynamicEncoder().to(device)
        self.localDynamicEncoder = DynamicEncoder().to(device)
        
        # encoder
        self.globalEncoder = GlobalEncoder().to(device)
        self.localEncoder = LocalEncoder().to(device)
        
    def forward(self, fullImages, cropImages, cropCenter):
        # extract center slice
        fullImages = fullImages[:,:,1]
        cropImages = cropImages[:,:,1]
    
        # encode input with each encoder
        self.globalInputFeature = torch.zeros(len(fullImages), 32*4, IMAGE_SIZE, IMAGE_SIZE)
        self.localInputFeature = torch.zeros(len(cropImages), 32*4, WINDOW_SIZE, WINDOW_SIZE)
        
        # T2WI Input
        self.globalInputFeature[:,0:32] = self.globalInputEncoder[0](torch.unsqueeze(fullImages[:,0], dim=1))
        self.localInputFeature[:,0:32] = self.localInputEncoder[0](torch.unsqueeze(cropImages[:,0], dim=1))
        # ADC Input
        self.globalInputFeature[:,32:64] = self.globalInputEncoder[1](torch.unsqueeze(fullImages[:,1], dim=1))
        self.localInputFeature[:,32:64] = self.localInputEncoder[1](torch.unsqueeze(cropImages[:,1], dim=1))
        # CE-T1WI Input
        self.globalInputFeature[:,64:96] = self.globalInputEncoder[2](torch.unsqueeze(fullImages[:,-1], dim=1))
        self.localInputFeature[:,64:96] = self.localInputEncoder[2](torch.unsqueeze(cropImages[:,-1], dim=1))
        # Dynamic T1WI Input
        self.globalInputFeature[:,96:128] = self.globalDynamicEncoder(torch.unsqueeze(fullImages[:,2:22], dim=1))
        self.localInputFeature[:,96:128] = self.localDynamicEncoder(torch.unsqueeze(cropImages[:,2:22], dim=1))

        # global encode
        self.cropFeature, self.gaussian = self.globalEncoder(self.globalInputFeature.to(device), cropCenter)
        # local encode
        self.output = self.localEncoder(torch.cat([self.localInputFeature, self.cropFeature], dim = 1).to(device))

        return self.gaussian, self.output

### Prepare

In [8]:
def saveOverlay(filename, pil_img, pil_map, color='jet', vmin=VMIN, vmax=VMAX):
    img = np.asarray(pil_img)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    map_img = np.asarray(pil_map)
    
    lim_img = np.where(map_img < vmin, 0, map_img)
    lim_img = ((lim_img - vmin) / (vmax-vmin) * 255).astype('uint8')
    
    heatmap = cv2.applyColorMap(lim_img, cv2.COLORMAP_JET)
    map_img = cv2.cvtColor(map_img, cv2.COLOR_GRAY2RGB)
    heatmap = np.where(map_img < vmin, 0, heatmap)
    
    superimposed_img = heatmap * 0.4 + img
    cv2.imwrite(filename, superimposed_img)
    
def saveImage(filename, pil_image):
    img = np.asarray(pil_image)
    cv2.imwrite(filename, img)

### Main Loop

In [9]:
# dataset
valid_dataset_fullImage = Dataset_FullImage(valid_files, valid_labels)
valid_dataset_cropImage = Dataset_CropImage(valid_files, valid_labels)

# for faster
torch.backends.cudnn.benchmark = True

# model
net = Net().to(device)
net.load_state_dict(torch.load(LOAD_DIR + LOAD_FILE))
net.train(False)

for data_num in range(int(len(valid_dataset_fullImage)/20)):
    with torch.no_grad():
        fullImage_info, fullImages, gaussianRoi = valid_dataset_fullImage[data_num*20]
        cropImage_info, cropImages = valid_dataset_cropImage[data_num*20]
        fullImages, cropImages = fullImages.to(device), cropImages.to(device)

        print(fullImage_info)

        output = torch.zeros(IMAGE_SIZE+WINDOW_SIZE-1, IMAGE_SIZE+WINDOW_SIZE-1)
        for i in range(len(cropImages)):
            for j in range(len(cropImages[0])):
                cropCenter = np.array([i*WINDOW_STRIDE, j*WINDOW_STRIDE], int)
                gaussian, prob = net(torch.unsqueeze(fullImages, dim=0), torch.unsqueeze(cropImages[i,j], dim=0), np.expand_dims(cropCenter,axis=0))
                cropCenter += int(WINDOW_SIZE/2)
                gaussian_prob = torch.zeros(WINDOW_SIZE, WINDOW_SIZE)
                # gaussian_prob[int(WINDOW_SIZE/2),int(WINDOW_SIZE/2)] = prob
                for k in range(WINDOW_SIZE):
                    for l in range(WINDOW_SIZE):
                        gaussian_prob[k,l] = prob * np.exp( -((int(WINDOW_SIZE/2)-k)**2 + (int(WINDOW_SIZE/2)-l)**2) / (2*((WINDOW_SIGMA+1)**2)) )
                
                output[cropCenter[0]-int(WINDOW_SIZE/2):cropCenter[0]+int(WINDOW_SIZE/2)+1, cropCenter[1]-int(WINDOW_SIZE/2):cropCenter[1]+int(WINDOW_SIZE/2)+1] += gaussian_prob

        output = output[int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE, int(WINDOW_SIZE/2):int(WINDOW_SIZE/2)+IMAGE_SIZE]
        output /= torch.max(output)
        pil_image = torchvision.transforms.functional.to_pil_image(output)
        pil_map = torchvision.transforms.functional.to_pil_image(fullImages[21,1])
        saveOverlay(SAVE_DIR + fullImage_info + '_pred.png', pil_map, pil_image)

        pil_map = torchvision.transforms.functional.to_pil_image(gaussianRoi)
        saveOverlay(SAVE_DIR + fullImage_info + '_ans.png', pil_map, pil_image)

        pil_image = torchvision.transforms.functional.to_pil_image(fullImages[0,1])
        saveImage(SAVE_DIR + fullImage_info + '_t2.png', pil_image)
        pil_image = torchvision.transforms.functional.to_pil_image(fullImages[1,1])
        saveImage(SAVE_DIR + fullImage_info + '_adc.png', pil_image)
        pil_image = torchvision.transforms.functional.to_pil_image(fullImages[2,1])
        saveImage(SAVE_DIR + fullImage_info + '_t1.png', pil_image)
        pil_image = torchvision.transforms.functional.to_pil_image(fullImages[21,1])
        saveImage(SAVE_DIR + fullImage_info + '_cet1.png', pil_image)

1_1_1
1_2_1
2_1_1
2_2_1
2_3_1
3_1_1
4_1_1
4_2_1
5_1_1
5_2_1
6_1_1
6_2_1
7_1_1
7_2_1
8_1_1
9_1_1
10_1_1
11_1_1
11_2_1
12_1_1
13_1_1
13_2_1
14_1_1
14_2_1
14_3_1
15_1_1
15_2_1
16_1_1
16_2_1
17_1_1
17_2_1
17_3_1
18_1_1
18_2_1
19_1_1
20_1_1
21_1_1
21_2_1
22_1_1
22_2_1
23_1_1
23_2_1
24_1_1
25_1_1
25_2_1
26_1_1
26_2_1
27_1_1
27_2_1
28_1_1
28_2_1
29_1_1
29_2_1
31_1_1
31_2_1
32_1_1
33_1_1
33_2_1
34_1_1
34_2_1
35_1_1
36_1_1
36_2_1
37_1_1
37_2_1
39_1_1
39_2_1
40_1_1
41_1_1
42_1_1
43_1_1
43_2_1
44_1_1
44_2_1
45_1_1
46_1_1
47_1_1
47_2_1
48_1_1
49_1_1
